In [1]:
import numpy as np
import math

In [2]:
from cardsample_problem import *

|S| = 45
|O| = 35


In [3]:
T = P.m*P.n+1
init_true_state= get_random_state()
#init_belief = init_belief_particle(all_states)
init_belief_hist = init_belief_particle_s0(all_states)
card_problem = CardProblem(init_true_state, init_belief_hist)

card_problem.agent.tree = None

print("*** Testing POMCP ***")

pomcp = pomdp_py.POMCP(max_depth=6, discount_factor=1.,
                           num_sims=20000, exploration_const=20,
                           rollout_policy=card_problem.agent.policy_model,
                           num_visits_init=1)

n_iter = 1
rewards = np.zeros(n_iter)

for it in range(n_iter):
    card_problem.agent.set_belief(init_belief_particle_s0(all_states), prior = True)

    r, tree_i = test_planner(card_problem, pomcp, nsteps=T)
    print(card_problem.agent.tree)

    rewards[it] = r

    card_problem.agent.tree = tree_i

print("average reward ", rewards.mean(), "over %d iter"%n_iter)

*** Testing POMCP ***
s0  State((1, 0, 0, 0, 0, 0), 0, None)
==== Step 1 ====
True state: State((1, 0, 0, 0, 0, 0), 0, None), 1 0
Action: Action(0)
Observation: Observation((1, 0, 0, 0)),  1
Reward: 1.0
True next state: State((1, 1, 1, 1, 0, 0), 1, 1), 2 0
{Observation((1, 0, 0, 0)): VNode(9829.000, 0.000, 9828 | dict_keys([Action(0), Action(1)])), Observation((0, 1, 0, 0)): VNode(9810.000, 0.000, 9809 | dict_keys([Action(0), Action(1)]))} State((1, 1, 1, 1, 0, 0), 1, 1) Observation((1, 0, 0, 0))
==== Step 2 ====
True state: State((1, 1, 1, 1, 0, 0), 1, 1), 2 0
Action: Action(1)
Observation: Observation((1, 0, 1, 0)),  2
Reward: 1.0
True next state: State((2, 1, 1, 1, 1, 1), 0, 1), 3 0
{Observation((1, 0, 0, 1)): VNode(9737.000, 0.000, 6669 | dict_keys([Action(0), Action(1)])), Observation((1, 0, 1, 0)): VNode(19427.000, 0.000, 13331 | dict_keys([Action(0), Action(1)]))} State((2, 1, 1, 1, 1, 1), 0, 1) Observation((1, 0, 1, 0))
==== Step 3 ====
True state: State((2, 1, 1, 1, 1, 1), 0, 

In [4]:
planner = pomcp

s0 = env_reset_s0(card_problem)
print("s0 ",   s0)
total_reward = 0
action = planner.plan(card_problem.agent)
#action = policy.sample(card_problem.agent.cur_belief)

true_state = copy.deepcopy(card_problem.env.state)
env_reward = card_problem.env.state_transition(action, execute=True)
true_next_state = copy.deepcopy(card_problem.env.state)
real_observation = card_problem.env.provide_observation(card_problem.agent.observation_model, action)
card_problem.agent.update_history(action, real_observation)    
    
    
print("True state: %s, %d %d" % (true_state, np.array(true_state.val)[::3].sum(), int(true_state.terminal)))
print("Action: %s" % str(action))
print("Observation: %s,  %d" % (str(real_observation), np.array(real_observation.val).sum()))
print("Reward: %s" % str(np.maximum(0, env_reward)))
print("True next state: %s, %d %d" % (true_next_state, 
    np.array(true_next_state.val)[::3].sum(), int(true_next_state.terminal)))

env_reward = card_problem.env.state_transition(action, execute=True)
true_next_state = copy.deepcopy(card_problem.env.state)
real_observation = card_problem.env.provide_observation(card_problem.agent.observation_model, action)
card_problem.agent.update_history(action, real_observation)    
    
    
print("True state: %s, %d %d" % (true_state, np.array(true_state.val)[::3].sum(), int(true_state.terminal)))
print("Action: %s" % str(action))
print("Observation: %s,  %d" % (str(real_observation), np.array(real_observation.val).sum()))
print("Reward: %s" % str(np.maximum(0, env_reward)))
print("True next state: %s, %d %d" % (true_next_state, 
    np.array(true_next_state.val)[::3].sum(), int(true_next_state.terminal)))

s0  State((1, 0, 0, 0, 0, 0), 0, None)
True state: State((1, 0, 0, 0, 0, 0), 0, None), 1 0
Action: Action(1)
Observation: Observation((0, 0, 0, 1)),  1
Reward: 0.0
True next state: State((1, 0, 0, 1, 1, 0), 1, 0), 2 0
True state: State((1, 0, 0, 0, 0, 0), 0, None), 1 0
Action: Action(1)
Observation: Observation((0, 0, 1, 1)),  2
Reward: 1.0
True next state: State((1, 0, 0, 2, 2, 1), 1, 1), 3 0


In [5]:
s0 = State(tuple(np.zeros(3*P.n, np.int32)), -1)
all_states = set()
create_all_states(s0, all_states)

In [6]:
all_states

{State((0, 0, 0, 1, 0, 0), 1, None),
 State((0, 0, 0, 2, 1, 1), 1, 1),
 State((0, 1, 0, 2, 0, 0), 1, 0),
 State((1, 0, 0, 0, 0, 0), 0, None),
 State((1, 0, 0, 1, 1, 0), 1, 0),
 State((1, 0, 0, 1, 1, 1), 0, 1),
 State((1, 0, 0, 2, 2, 1), 1, 1),
 State((1, 0, 0, 2, 2, 2), 0, 1),
 State((1, 1, 0, 1, 0, 0), 0, 0),
 State((1, 1, 0, 2, 1, 0), 1, 0),
 State((1, 1, 0, 2, 1, 1), 0, 1),
 State((1, 1, 1, 1, 0, 0), 1, 1),
 State((1, 1, 1, 2, 1, 1), 1, 1),
 State((1, 2, 0, 2, 0, 0), 0, 0),
 State((1, 2, 1, 2, 0, 0), 1, 0),
 State((2, 0, 0, 0, 1, 0), 0, 0),
 State((2, 0, 0, 1, 2, 0), 1, 0),
 State((2, 0, 0, 1, 2, 1), 0, 1),
 State((2, 0, 0, 2, 3, 1), 1, 1),
 State((2, 0, 0, 2, 3, 2), 0, 1),
 State((2, 0, 0, 2, 4, 2), $, 1),
 State((2, 1, 0, 1, 1, 0), 0, 0),
 State((2, 1, 0, 2, 2, 0), 1, 0),
 State((2, 1, 0, 2, 2, 1), 0, 1),
 State((2, 1, 0, 2, 3, 1), $, 1),
 State((2, 1, 1, 0, 0, 0), 0, 1),
 State((2, 1, 1, 1, 1, 0), 1, 0),
 State((2, 1, 1, 1, 1, 1), 0, 1),
 State((2, 1, 1, 2, 2, 1), 1, 1),
 State((

In [7]:
(P.n*2)**(P.n*P.m)

256

In [8]:
len(all_states)

45

In [9]:
# states for n=5;m=2 is 330438

In [10]:
for s in all_states:
    val = np.array(s.val)
    if (val[::3].sum()-1 != val[1::3].sum()) and s.card != "$" :
        print(s, val[::3].sum(), val[1::3].sum(),"error in states actions")

In [11]:
all_obs = create_all_observations(all_states)

In [12]:
len(all_obs)

35

In [15]:
rewards_all = np.load("rewards_pomcp.npy")

In [18]:
num = (rewards_all != 0).sum()

In [19]:
rewards_all.sum() * 1.0 / num

3.772727272727273

In [20]:
rewards_all

array([3., 2., 5., 3., 3., 6., 6., 4., 3., 2., 6., 2., 5., 6., 4., 3., 3.,
       3., 5., 4., 3., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [13]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

from julia.QMDP import QMDPSolver
from julia.SARSOP import SARSOPSolver

ModuleNotFoundError: No module named 'julia'